<a href="https://colab.research.google.com/github/marcelounb/Deep_Learning_with_python_JasonBrownlee/blob/master/16_3_Reduce_Over%EF%AC%81tting_With_Dropout_on_the_visible_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A simple and powerful regularization technique for neural networks and deep learning models is dropout. In this lesson you will discover the dropout regularization technique and how to apply it to your models in Python with Keras. After completing this lesson you will know:
1. How the dropout regularization technique works.
2. How to use dropout on your input and hidden layers.
3. How to use dropout on your hidden layers

#Dropout Regularization For Neural Networks
Dropout is a regularization technique for neural network models proposed by Srivastava, et al. in their 2014 paper Dropout: A Simple Way to Prevent Neural Networks from Overﬁtting. 

Dropout is a technique where randomly selected neurons are ignored during training. They are dropped-out randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass. 

As a neural network learns, neuron weights settle into their context within the network. Weights of neurons are tuned for speciﬁc features providing some specialization. Neighboring neurons become to rely on this specialization, which if taken too far can result in a fragile model too specialized to the training data. 

This reliant on context for a neuron during training is referred to as complex co-adaptations. You can imagine that if neurons are randomly dropped out of the network during training, that other neurons will have to step in and handle the representation required to make predictions for the missing neurons. This is believed to result in multiple independent internal representations being learned by the network. 

The effect is that the network becomes less sensitive to the speciﬁc weights of neurons. This in turn results in a network that is capable of better generalization and is less likely to overﬁt the training data.


# Dropout Regularization in Keras

Dropout is easily implemented by randomly selecting nodes to be dropped-out with a given probability (e.g. 20%) each weight update cycle. This is how Dropout is implemented in Keras. 

Dropout is only used during the training of a model and is not used when evaluating the skill of the model. Next we will explore a few different ways of using Dropout in Keras. 

The examples will use the **Sonar dataset** binary classiﬁcation dataset (learn more in Section 11.1). We will evaluate the developed models using scikit-learn with 10-fold cross validation, in order to better tease out differences in the results. 

There are 60 input values and a single output value and the input values are standardized before being used in the network. The baseline neural network model has two hidden layers, the ﬁrst with 60 units and the second with 30. Stochastic gradient descent is used to train the model with a relatively low learning rate and momentum.

In [1]:
# Baseline Model on the Sonar Dataset 
import numpy as np
import pandas as pd
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.constraints import maxnorm 
from keras.optimizers import SGD 
from sklearn.model_selection import cross_val_score 
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import StratifiedKFold 
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
# load dataset 
dataframe = pd.read_csv("sonar.csv", header=None) 
dataset = dataframe.values 
# split into input (X) and output (Y) variables 
X = dataset[:,0:60].astype(float) 
Y = dataset[:,60] 

In [3]:
# fix random seed for reproducibility 
seed = 7 
np.random.seed(seed)

In [4]:
# encode class values as integers 
encoder = LabelEncoder() 
encoder.fit(Y) 
encoded_Y = encoder.transform(Y)

In [5]:
# baseline 
"""def create_baseline(): 
  # create model 
  model = Sequential() 
  model.add(Dense(60, input_dim=60, kernel_initializer= 'normal' , activation= 'relu' )) 
  model.add(Dense(30, kernel_initializer= 'normal' , activation= 'relu' )) 
  model.add(Dense(1, kernel_initializer= 'normal' , activation= 'sigmoid' )) 
  # Compile model 
  sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False) 
  model.compile(loss= 'binary_crossentropy' , optimizer='sgd', metrics=[ 'accuracy' ]) 
  return model"""

"def create_baseline(): \n  # create model \n  model = Sequential() \n  model.add(Dense(60, input_dim=60, kernel_initializer= 'normal' , activation= 'relu' )) \n  model.add(Dense(30, kernel_initializer= 'normal' , activation= 'relu' )) \n  model.add(Dense(1, kernel_initializer= 'normal' , activation= 'sigmoid' )) \n  # Compile model \n  sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False) \n  model.compile(loss= 'binary_crossentropy' , optimizer='sgd', metrics=[ 'accuracy' ]) \n  return model"

# Using Dropout on the Visible Layer

Dropout can be applied to input neurons called the visible layer. In the example below we add a new Dropout layer between the input (or visible layer) and the ﬁrst hidden layer. 

The dropout rate is set to 20%, meaning one in ﬁve inputs will be randomly excluded from each update cycle. Additionally, as recommended in the original paper on dropout, a constraint is imposed on the weights for each hidden layer, ensuring that the maximum norm of the weights does not exceed a value of 3. This is done by setting the W constraint argument on the Dense class when constructing the layers. 

The learning rate was lifted by one order of magnitude and the momentum was increased to 0.9. These increases in the learning rate were also recommended in the original dropout paper. Continuing on from the baseline example above, the code below exercises the same network with input dropout.


In [10]:
# dropout in the input layer with weight constraint 
def create_model(): 
  # create model 
  model = Sequential() 
  model.add(Dropout(0.2, input_shape=(60,))) 
  model.add(Dense(60, kernel_initializer= 'normal' , activation= 'relu' , kernel_constraint=maxnorm(3))) 
  model.add(Dense(30, kernel_initializer= 'normal' , activation= 'relu' , kernel_constraint=maxnorm(3))) 
  model.add(Dense(1, kernel_initializer= 'normal' , activation= 'sigmoid' )) 
  # Compile model 
  sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False) 
  model.compile(loss= 'binary_crossentropy' , optimizer=sgd, metrics=[ 'accuracy' ]) 
  return model

In [11]:
np.random.seed(seed) 
estimators = [] 
estimators.append(( 'standardize' , StandardScaler())) 
estimators.append(( 'mlp' , KerasClassifier(build_fn=create_model, epochs=300, batch_size=16, verbose=0))) 
pipeline = Pipeline(estimators)

In [12]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed) 
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold) 
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 84.55% (7.27%)


Running the example with dropout in the visible layer provides a nice lift in classiﬁcation accuracy to 86%
